### MUIZ ALVI
#### 263791 

In [27]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [4]:
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

### Reading EDF files

In [5]:
GOOGLE_COLAB = True
path = ""
if GOOGLE_COLAB:
  from google.colab import drive, files
  drive.mount('/content/drive/')
  path = "/content/drive/My Drive/Colab Notebooks/dataset_edf/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
!pip install -U mne
import mne
sequences = list()
for i in range(1,11):
    file_path = path + 'file_' + str(i) + '.edf'
    data = mne.io.read_raw_edf(file_path)
    values = data.get_data()
    sequences.append(values)

targets = ['normal', 'normal', 'normal', 'normal', 'normal', 'abnormal', 'abnormal', 'abnormal', 'abnormal', 'abnormal']

Requirement already up-to-date: mne in /usr/local/lib/python3.7/dist-packages (0.22.0)
Extracting EDF parameters from /content/drive/My Drive/Colab Notebooks/dataset_edf/file_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/drive/My Drive/Colab Notebooks/dataset_edf/file_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/drive/My Drive/Colab Notebooks/dataset_edf/file_3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/drive/My Drive/Colab Notebooks/dataset_edf/file_4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/drive/My Drive/Colab Notebooks/dataset_edf/file_5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting ED

In [7]:
sequences[0]

array([[-5.28216654e-08, -2.60993696e-08, -2.88606226e-09, ...,
         2.90972763e-11, -7.90661479e-09, -1.96832685e-09],
       [-1.78937063e-07, -3.51408469e-08, -3.58324254e-09, ...,
        -7.97777475e-08,  1.13971142e-08,  4.07494884e-08],
       [ 5.68357801e-08, -8.60206409e-09, -2.64540699e-08, ...,
        -9.29454751e-08, -2.27666064e-08, -1.09433107e-08],
       ...,
       [-3.61376883e-08,  6.07795285e-08,  8.62899223e-08, ...,
        -1.28320443e-07, -3.89783653e-08,  1.75566778e-08],
       [-8.29103243e-09, -6.83575502e-09, -6.58910903e-10, ...,
        -1.15573217e-08, -2.38139915e-08, -3.08316625e-08],
       [-8.29103243e-09, -6.83575502e-09, -6.58910903e-10, ...,
        -1.15573217e-08, -2.38139915e-08, -3.08316625e-08]])

### Preprocessing Data

In [8]:
for i in range(0, 10):
  [row,col] = sequences[i].shape
  print(sequences[i].shape)


(21, 224000)
(21, 190500)
(21, 195000)
(21, 202750)
(21, 195250)
(21, 149250)
(21, 162500)
(21, 211000)
(21, 196250)
(21, 171250)


In [9]:
len_sequences = []
for i in range(0,10):
  for j in range(0,21):
    len_sequences.append(len(sequences[i][j]))
pd.Series(len_sequences).describe()

count       210.000000
mean     189775.000000
std       21543.927668
min      149250.000000
25%      171250.000000
50%      195125.000000
75%      202750.000000
max      224000.000000
dtype: float64

In [10]:
#Padding the sequence with the values in last coloumn to max length
new_seq = np.zeros((10, 21, 224000))
to_pad = 224000
for i in range(0,10):
  for j in range(0,21):
    len_one_seq = len(sequences[i][j])
    last_val = sequences[i][j][-1]
    n = to_pad - len_one_seq
    new_seq [i][j] = np.concatenate([sequences[i][j], np.zeros(n)])

In [11]:
for i in range(0, 10):
  [row,col] = new_seq[i].shape
  print(new_seq[i].shape)

(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)
(21, 224000)


In [12]:
new_seq.shape

(10, 21, 224000)

In [31]:
final_seq = new_seq.reshape((392, 60, 2000))  

In [32]:
final_seq.shape

(392, 60, 2000)

In [ ]:
path = "/content/drive/My Drive/Colab Notebooks/dataset_edf/"
for i in range(0,196):
    file_path = path + 'normal.' + str(i) + '.edf'
    data = mne.io.read_raw_edf(file_path)
    values = data.get_data()
    sequences.append(values)

In [ ]:
# Organize data into train, valid, test dirs

os.chdir('abnormal-vs-normal')
if os.path.isdir('train/abnormal') is False:
    os.makedirs('train/abnormal')
    os.makedirs('train/normal')
    os.makedirs('valid/abnormal')
    os.makedirs('valid/normal')
    os.makedirs('test/abnormal')
    os.makedirs('test/normal')
    
    for c in random.sample(glob.glob('normal*'), 120):
        shutil.move(c, 'train/normal')
    for c in random.sample(glob.glob('abnormal*'), 120):
        shutil.move(c, 'train/abnormal')
    for c in random.sample(glob.glob('normal*'), 40):
        shutil.move(c, 'valid/normal')
    for c in random.sample(glob.glob('abnormal*'), 40):
        shutil.move(c, 'valid/abnormal')
    for c in random.sample(glob.glob('normal*'), 36):
        shutil.move(c, 'test/normal')
    for c in random.sample(glob.glob('abnormal*'), 36):
        shutil.move(c, 'test/abnormal')
    
os.chdir('../../')

#### Building LSTM for preprocessing

In [33]:
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (60, 2000)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 1))

In [ ]:
train_path = '/content/drive/My Drive/Colab Notebooks/dataset_edf/abnormal-vs-normal/train/'
valid_path = '/content/drive/My Drive/Colab Notebooks/dataset_edf/abnormal-vs-normal/valid/'
test_path = '/content/drive/My Drive/Colab Notebooks/dataset_edf/abnormal-vs-normal/test/'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function = model.preprocess_input) \
    .flow_from_directory(directory = train_path, target_size = (60, 2000), classes = ['normal', 'abnormal'], batch_size = 10)
valid_batches = ImageDataGenerator(preprocessing_function = model.preprocess_input) \
    .flow_from_directory(directory = valid_path, target_size = (60, 2000), classes = ['normal', 'abnormal'], batch_size = 10)
test_batches = ImageDataGenerator(preprocessing_function = model.preprocess_input) \
    .flow_from_directory(directory = test_path, target_size = (60, 2000), classes = ['normal', 'abnormal'], batch_size = 10, shuffle = False)